In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import random
from sklearn.utils import shuffle

In [8]:
def check_correct(predict, y):
    result = {}
    result['cancer-correct'] = 0
    result['cancer-wrong'] = 0
    result['normal-correct'] = 0
    result['normal-wrong'] = 0

    for i in range(len(predict)) :
        if predict[i] == y[i] :
            if y[i] == 0 :
                result['normal-correct'] += 1
            else :
                result['cancer-correct'] += 1
        else :
            if y[i] == 0 :
                result['normal-wrong'] += 1
            else :
                result['cancer-wrong'] += 1

    #for result_k, result_v in result.items():
        #print(result_k +" : "+ str(result_v))
    sensitivity=result['cancer-correct']/(result['cancer-correct']+result['cancer-wrong'])
    specificity=result['normal-correct']/(result['normal-correct']+result['normal-wrong'])
    #print("Sensitivity :", sensitivity)
    #print("Specificity :", specificity)
    return sensitivity, specificity

In [ ]:
path = "C:/test/coupled/"
data1 = pd.read_csv(path+"TCGA_GTEx_foundation_308_uncoupled_cancer.csv")
data0 = pd.read_csv(path+"TCGA_GTEx_foundation_308_uncoupled_normal.csv")
data = pd.concat([data0,data1])
data = data.sample(frac=1)
x = data.drop(["patient","cancer_code","result"],axis=1)
y = data.result
x,y = shuffle(x, y, random_state = 0)

In [32]:
data1_c = pd.read_csv(path+"TCGA_GTEx_foundation_308_coupled_unique_cancer.csv")
data0_c = pd.read_csv(path+"TCGA_GTEx_foundation_308_coupled_unique_normal.csv")
data1_c = data1_c.sample(frac = 1)
x1_c = data1_c.drop(["patient","cancer_code","result"],axis=1)
y1_c = data1_c.result
data0_c = data0_c.sample(frac = 1)
x0_c = data0_c.drop(["patient","cancer_code","result"],axis=1)
y0_c = data0_c.result
data_c = pd.concat([data0_c,data1_c])
data_c = data_c.sample(frac=1)
x_c = data_c.drop(["patient","cancer_code","result"],axis=1)
y_c = data_c.result

In [33]:
print(data1.shape)
print(data0.shape)
print(data.shape)

(8023, 311)
(7240, 311)
(15263, 311)


In [29]:
from keras.layers import Input, Dense
from keras.models import Model

input_m1 = Input(shape=(308,))
h1_m1 = Dense(200,activation='relu')(input_m1)
h2_m1 = Dense(150,activation='relu')(h1_m1) 
h3_m1 = Dense(100,activation='relu')(h2_m1) 
h4_m1 = Dense(10,activation='relu')(h3_m1) 
output_m1 = Dense(1,activation="sigmoid")(h4_m1) 
model1 = Model(inputs=input_m1,outputs=output_m1)

In [30]:
model1.compile(optimizer='Adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])
model1.fit(x,  y, epochs=10)

#model1.save('my_model.h5')

Epoch 1/10
15263/15263 [==============================] - 3s 207us/step - loss: 0.0441 - acc: 0.9855
Epoch 2/10
15263/15263 [==============================] - 2s 157us/step - loss: 0.0131 - acc: 0.9959
Epoch 3/10
15263/15263 [==============================] - 2s 159us/step - loss: 0.0106 - acc: 0.9969
Epoch 4/10
15263/15263 [==============================] - 2s 159us/step - loss: 0.0090 - acc: 0.9973
Epoch 5/10
15263/15263 [==============================] - 2s 157us/step - loss: 0.0083 - acc: 0.9973
Epoch 6/10
15263/15263 [==============================] - 2s 157us/step - loss: 0.0085 - acc: 0.9972
Epoch 7/10
15263/15263 [==============================] - 2s 158us/step - loss: 0.0052 - acc: 0.9984
Epoch 8/10
15263/15263 [==============================] - 2s 161us/step - loss: 0.0080 - acc: 0.9973
Epoch 9/10
15263/15263 [==============================] - 3s 170us/step - loss: 0.0032 - acc: 0.9991
Epoch 10/10
15263/15263 [==============================] - 2s 158us/step - loss: 0.0051 - a

In [35]:
print(model1.evaluate(x1_c, y1_c))
print(model1.evaluate(x0_c, y0_c))

600/600 [==============================] - 0s 300us/step
[0.0012388796868617645, 1.0]
600/600 [==============================] - 0s 52us/step
[3.742867234547933, 0.255]


In [22]:
predict_y = np.where(model1.predict(x) > 0.5, 1, 0).flatten()
predict_y_c = np.where(model1.predict(x_c) > 0.5, 1, 0).flatten()
print(check_correct(predict_y, list(y)))
print(check_correct(predict_y_c, list(y_c)))

(0.9588682537704101, 0.9259668508287293)
(0.99, 0.9833333333333333)


In [14]:
y

4800    0
6050    1
533     0
6809    1
5240    0
4868    0
894     1
7531    1
4094    0
6505    0
4331    0
3387    0
4595    1
3086    0
2284    1
2648    1
654     0
1245    1
85      1
4674    0
7777    1
2252    1
7472    1
727     0
7205    1
19      1
4469    1
1205    0
2760    1
4509    0
       ..
6072    0
1136    0
5740    1
4034    0
2194    1
4231    0
5313    0
2745    0
4673    1
2172    0
6560    1
6039    0
5999    1
6851    0
308     0
1260    0
3725    1
3658    0
4694    1
4746    1
3645    0
3967    1
3875    1
1222    1
7222    1
4822    0
6599    0
6992    0
188     1
2791    0
Name: result, Length: 15263, dtype: int64